## Patient Information Objects

### Patient

In [1]:
from models.patient import Patient

In [2]:
p1 = Patient('2712205f-755e-4897-acb3-926895b7d635')

In [3]:
p1.FIRST

'Emory494'

In [4]:
p1.get_age()

datetime.timedelta(days=29946, seconds=84223, microseconds=680434)

### Allergy

## Text generation